## Download and clean up near-real-time Wave Glider data

* read in data
* sort time and remove redundant records
* add lat/lon to ADCP files
* add depth vector to ADCP files
* save cleaned up ADCP files

first cut by Tom, 10/18/2021  
Updated for IOP1, 10/9/2022
Updated version for IOP2, ~4/10/2023

In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import cftime
import requests
import cartopy.crs as ccrs                   # import projections
import cartopy
import gsw
import functions  # requires functions.py from this directory

In [2]:
# %matplotlib inline
%matplotlib qt5
plt.rcParams['figure.figsize'] = (7,4)
plt.rcParams['figure.dpi'] = 200
plt.rcParams['savefig.dpi'] = 400
plt.close('all')

__figdir__ = '../plots/' 
savefig_args = {'bbox_inches':'tight', 'pad_inches':0.2}
plotfiletype='png'

In [3]:
savefig = True
zoom = True
if zoom:
    xmin, xmax = (-127,-121)
    ymin, ymax = (36.25,38.5)
    levels = np.linspace(14,17,21)-2.5
else:
    xmin, xmax = (-127,-121)
    ymin, ymax = (35, 41)
    levels = np.linspace(13,18,11)

    

In [4]:
#!conda install -y -c conda-forge netcdf4 h5netcdf scipy pydap zarr fsspec cftime rasterio cfgrib pooch

In [5]:
# List of files
input_list = [f"{i:0>3d}" for i in range(98,121)] 
url_prefix = 'http://smode.whoi.edu:8080/thredds/fileServer/IOP2_2023/Sally_Ride/ADCP/300kHz/wh300'
postfix = '_compr.nc'#PLD2_TAB1
outpath='../data/raw/SR_NRT_IOP2/wh300_'

#http://smode.whoi.edu:8080/thredds/dodsC/IOP2_2023/waveglider/KELVIN_23.nc.html
#http://smode.whoi.edu:8080/thredds/fileServer/IOP2_2023/waveglider/KELVIN_23.nc

In [6]:
# Reading the files over the internet directly doesn't always work well
# Download instead

n=0
file_list1 = []
for input_file in input_list:
    outfile1 = outpath+input_file+postfix
    # Read and save files
    url1 = url_prefix+input_file+postfix
    '''
    # Right now, I am manually downloading
    file_data = requests.get(url1).content
    # create the file in write binary mode, because the data we get from net is in binary
    with open(outfile1, "wb") as file:
        file.write(file_data)
    n=n+1
    print(url1)
    '''
    file_list1.append(outfile1)


In [7]:
file_list1

['../data/raw/SR_NRT_IOP2/wh300_098_compr.nc',
 '../data/raw/SR_NRT_IOP2/wh300_099_compr.nc',
 '../data/raw/SR_NRT_IOP2/wh300_100_compr.nc',
 '../data/raw/SR_NRT_IOP2/wh300_101_compr.nc',
 '../data/raw/SR_NRT_IOP2/wh300_102_compr.nc',
 '../data/raw/SR_NRT_IOP2/wh300_103_compr.nc',
 '../data/raw/SR_NRT_IOP2/wh300_104_compr.nc',
 '../data/raw/SR_NRT_IOP2/wh300_105_compr.nc',
 '../data/raw/SR_NRT_IOP2/wh300_106_compr.nc',
 '../data/raw/SR_NRT_IOP2/wh300_107_compr.nc',
 '../data/raw/SR_NRT_IOP2/wh300_108_compr.nc',
 '../data/raw/SR_NRT_IOP2/wh300_109_compr.nc',
 '../data/raw/SR_NRT_IOP2/wh300_110_compr.nc',
 '../data/raw/SR_NRT_IOP2/wh300_111_compr.nc',
 '../data/raw/SR_NRT_IOP2/wh300_112_compr.nc',
 '../data/raw/SR_NRT_IOP2/wh300_113_compr.nc',
 '../data/raw/SR_NRT_IOP2/wh300_114_compr.nc',
 '../data/raw/SR_NRT_IOP2/wh300_115_compr.nc',
 '../data/raw/SR_NRT_IOP2/wh300_116_compr.nc',
 '../data/raw/SR_NRT_IOP2/wh300_117_compr.nc',
 '../data/raw/SR_NRT_IOP2/wh300_118_compr.nc',
 '../data/raw

In [8]:
def fix_ds_time(ds):
    '''
    Drop nonunique values in realtime data files and sort time.
    
    Input: ds, xarray dataset
    Output: ds, xarray dataset
    '''
    t, ind = np.unique(ds.time, return_index=True)
    ds2 = ds.isel(time=ind,drop=True)
    return ds2

In [9]:
file_list1[0]

'../data/raw/SR_NRT_IOP2/wh300_098_compr.nc'

In [10]:
ds_adcp = xr.open_mfdataset(file_list1,decode_times=True)

In [11]:
ds_adcp

<xarray.Dataset>
Dimensions:     (time: 15035, depth_cell: 50)
Coordinates:
  * time        (time) datetime64[ns] 2023-04-09T03:52:15 ... 2023-05-01T13:3...
Dimensions without coordinates: depth_cell
Data variables: (12/14)
    trajectory  (time) int32 -970446667 -970446667 ... 1980983066 1980983066
    lon         (time) float64 dask.array<chunksize=(599,), meta=np.ndarray>
    lat         (time) float64 dask.array<chunksize=(599,), meta=np.ndarray>
    depth       (time, depth_cell) float32 dask.array<chunksize=(599, 50), meta=np.ndarray>
    u           (time, depth_cell) float32 dask.array<chunksize=(599, 50), meta=np.ndarray>
    v           (time, depth_cell) float32 dask.array<chunksize=(599, 50), meta=np.ndarray>
    ...          ...
    pflag       (time, depth_cell) uint8 dask.array<chunksize=(599, 50), meta=np.ndarray>
    heading     (time) float32 dask.array<chunksize=(599,), meta=np.ndarray>
    tr_temp     (time) float32 dask.array<chunksize=(599,), meta=np.ndarray>
    num_pings   (time) int16 dask.array<chunksize=(599,), meta=np.ndarray>
    uship       (time) float32 dask.array<chunksize=(599,), meta=np.ndarray>
    vship       (time) float32 dask.array<chunksize=(599,), meta=np.ndarray>
Attributes:
    featureType:   trajectoryProfile
    history:       Created: 2023-04-09 23:50:44 UTC
    Conventions:   COARDS CF-1.6
    software:      pycurrents
    hg_changeset:  3306:670a5d9ffffd
    title:         Shipboard ADCP velocity profiles
    description:   Shipboard ADCP velocity profiles from SR2307 using instrum...
    cruise_id:     SR2307
    sonar:         wh300
    yearbase:      2023
    ship_name:     Sally Ride

In [12]:
fout_adcp = outpath + 'SR_ADCP_clean.nc'
ds_adcp = ds_adcp.rename(name_dict={'v':'current_north', 'u':'current_east','lon':'Longitude','lat':'Latitude'})
ds_adcp.to_netcdf(fout_adcp)
#ds.Longitude[tind].values,ds.Latitude[tind].values,np.squeeze(ds.current_east[tind, zind].values

In [13]:
vmin = -0.5
vmax = 0.5
fig = plt.figure()
plt.set_cmap(cmap=plt.get_cmap('turbo'))
# ax1 = plt.subplot(len(WG_list),1,len(WG_list))
# ax1.set_xlim(tmin,tmax)
ds = ds_adcp
im = plt.pcolor(ds.time.values,ds.depth[-1,:].values,ds.current_north.transpose(),vmin=vmin,vmax=vmax)
# plt.contourf(ds.time.values,ds.z_matrix[:,1],ds.current_east,levels)
#plt.ylim(-60, 0)
plt.title(' SR vel')
fig=plt.gcf()
fig.autofmt_xdate()


In [14]:
fout_adcp

'../data/raw/SR_NRT_IOP2/wh300_SR_ADCP_clean.nc'